# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import re

from sklearn.datasets import make_multilabel_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
def load_data():
    # load data from database
    engine = create_engine('sqlite:///DisasterDB_ETL.db')
    conn = engine.connect()
    df = pd.read_sql_table("DisasterETL_DB", conn)
    X = df['message'] #only the message text, and not others.
    Y= df.drop(['id','genre','message','original'],axis=1)
    category_names = list(Y.columns) #Y should only contain category colunm
    X_head = X.head(5)
    Y_head = Y.head(5)
    
    return X, Y, category_names 


In [3]:
X, y, category_names = load_data()


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens



### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            print(len(sentence_list))
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return 1
        return 0
    
    """def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False
    """
    def fit(self, x, y=None):
        return self

          
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)



In [11]:
def build_model():
    bootstrap_p = [True, False]
    #KNN_model = KNeighborsClassifier(n_neighbors=5)
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('starting_verb', StartingVerbExtractor())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])   
    pl = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        #('clf',MultiOutputClassifier( RandomForestClassifier(bootstrap = bootstrap_p)))
        #('clf',MultiOutputClassifier( KNN_modelAdaBoostClassifier)) # accr = 0.73
       ('clf',MultiOutputClassifier( AdaBoostClassifier())) # accu = 0.75
    ])

    return pipeline



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    #confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    #print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)


In [13]:
X, y, category_names  = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = build_model()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

display_results(y_test, y_pred)
print("model score: %.3f" % model.score(X_test, y_test))


3
3
3
1
2
2
1
1
1
1
1
2
2
4
4
4
4
1
1
1
1
2
2
1
1
1
2
2
1
3
3
3
1
2
2
3
3
3
1
1
1
1
1
1
1
1
1
1
1
1
2
2
5
5
5
5
5
1
1
3
3
3
1
1
1
1
2
2
1
1
1
1
4
4
4
4
2
2
1
1
4
4
4
4
1
1
1
1
1
2
2
1
1
2
2
1
1
3
3
1
1
1
1
1
1
1
1
2
2
1
1
2
2
1
1
1
1
1
1
2
2
1
1
1
1
1
3
3
3
1
1
1
3
3
3
4
4
4
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
3
3
3
3
3


IndexError: list index out of range

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:

print(classification_report(y_test, y_pred, target_names=category_names))

In [13]:
model.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7feb9db95c80>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
n_estimators = [int(x) for x in np.linspace(start = 5 , stop = 15, num = 10)] # returns 10 numbers 

max_features = ['auto', 'log2']

max_depth = [int(x) for x in np.linspace(5, 10, num = 2)] 

max_depth.append(None)


parameters = {'clf__estimator__n_estimators': n_estimators,

               'clf__estimator__max_features': max_features,

               'clf__estimator__max_depth': max_depth}

X, y, category_names  = load_data()
X_train, X_test, y_train, y_test = train_test_split(X, y)

model = build_model()
cv = GridSearchCV(model, param_grid =parameters )
model.fit(X_train, y_train)
y_pred = model.predict(X_test)



### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
print(classification_report(y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.77      0.93      0.84      4964
               request       0.43      0.07      0.11      1103
                 offer       0.00      0.00      0.00        30
           aid_related       0.46      0.19      0.27      2697
          medical_help       0.05      0.00      0.00       484
      medical_products       0.00      0.00      0.00       328
     search_and_rescue       0.00      0.00      0.00       193
              security       0.00      0.00      0.00       109
              military       0.00      0.00      0.00       204
           child_alone       0.00      0.00      0.00         0
                 water       0.11      0.01      0.01       390
                  food       0.09      0.00      0.01       752
               shelter       0.10      0.01      0.01       588
              clothing       0.00      0.00      0.00       114
                 money       0.00      

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [15]:
display_results(y_test, y_pred)

Labels: [0 1]
Accuracy: related                   0.734910
request                   0.827215
offer                     0.995392
aid_related               0.571187
medical_help              0.922593
medical_products          0.947166
search_and_rescue         0.969283
security                  0.982952
military                  0.968515
child_alone               1.000000
water                     0.936876
food                      0.881585
shelter                   0.904777
clothing                  0.982338
money                     0.974044
missing_people            0.987713
refugees                  0.967593
death                     0.954385
other_aid                 0.857318
infrastructure_related    0.930272
transport                 0.954999
buildings                 0.948241
electricity               0.980802
tools                     0.994932
hospitals                 0.987252
shops                     0.994624
aid_centers               0.986024
other_infrastructure      0.956

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.